In [23]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

In [24]:
train_df = pd.read_csv("assets/preprocessed_train.csv")
train_df

,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,0.352151,3,-0.216204,-0.395204,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,0.133379,8,4,208500
1,1,-5.199338,3,0.587964,0.079120,1,3,3,0,2,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.493553,-0.419741,8,4,181500
2,2,0.352151,3,-0.096753,0.630613,1,0,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.924252,0.133379,8,4,223500
3,3,0.723343,3,-0.526417,0.030279,1,0,3,0,0,...,2.449513,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,-5.199338,8,0,140000
4,4,0.352151,3,0.761353,1.348041,1,0,3,0,2,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,5.199338,0.133379,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,1455,0.352151,3,-0.344154,-0.585509,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,0.697507,-0.419741,8,4,175000
1090,1456,-5.199338,3,0.835558,1.107339,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.562721,5.199338,8,4,210000
1091,1457,0.723343,3,-0.149853,-0.137177,1,3,3,0,4,...,-5.199338,-5.199338,-5.199338,-5.199338,5.199338,-0.493553,5.199338,8,4,266500
1092,1458,-5.199338,3,-0.096753,0.118777,1,3,3,0,4,...,1.295363,-5.199338,-5.199338,-5.199338,-5.199338,-0.860725,5.199338,8,4,142125


In [25]:
train_df.drop(columns=["Unnamed: 0"], inplace=True)
y = train_df["SalePrice"]
X = train_df.drop(columns=["SalePrice"])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
y_train

6       307000
789     180000
1049    193000
837     117000
661     200500
         ...  
330      90350
466     118500
121     215000
1044    281213
860     174500
Name: SalePrice, Length: 875, dtype: int64

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the improved regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  # Increased neurons
        self.hidden2 = nn.Linear(20, 10)  # Additional hidden layer
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Hyperparameters (updated)
l1_lambda = 0.001
l2_lambda = 0.001
learning_rate = 0.001
epochs = 500  # More training

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with GPU
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train_torch)
    
    # Compute base MSE loss
    mse_loss = criterion(y_pred, y_train_torch)
    rmse_loss = torch.sqrt(mse_loss)

    # Compute L1 and L2 loss
    l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
    l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())

    # Total loss with regularization
    # total_loss = mse_loss + l1_lambda * l1_loss + l2_lambda * l2_loss

    # Backpropagation
    rmse_loss.backward()
    optimizer.step()

    if epoch % 50 == 0:  # Less frequent logging
        print(f"Epoch {epoch}, RMSE: {rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Using device: cpu


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler



Using device: cpu
Epoch 0, Train RMSE: 48829.39453125
Epoch 50, Train RMSE: 2409.87939453125
Epoch 100, Train RMSE: 648.390380859375
Epoch 150, Train RMSE: 176.7240447998047
Epoch 200, Train RMSE: 120.6166000366211
Test RMSE: 55847.9921875


In [48]:
# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 250
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        # total_loss = mse_loss + l1_lambda * l1_loss + l2_lambda * l2_loss
        
        rmse_loss.backward()  # Compute gradients
        return rmse_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Epoch 0, Train RMSE: 204115.375
Epoch 50, Train RMSE: 1024.5865478515625
Epoch 100, Train RMSE: 183.5473175048828
Epoch 150, Train RMSE: 144.09107971191406
Epoch 200, Train RMSE: 106.96619415283203
Test RMSE: 60765.32421875


In [44]:
# Using L1 norm


# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 100
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        total_loss = mse_loss + l1_lambda * l1_loss
        
        total_loss.backward()  # Compute gradients
        return total_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Epoch 0, Train RMSE: 204115.59375
Epoch 50, Train RMSE: 629.6298828125
Test RMSE: 54197.40234375


In [47]:
# Using L2 norm


# Normalize input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

# Convert Pandas DataFrames to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Define the regression model
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, 20)  
        self.hidden2 = nn.Linear(20, 10)  
        self.output = nn.Linear(10, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.output(x)
        return x

# Define model and move it to GPU
input_size = X_train_torch.shape[1]
model = RegressionNN(input_size).to(device)

# Define loss function
criterion = nn.MSELoss()

# Use L-BFGS optimizer
optimizer = optim.LBFGS(model.parameters(), lr=0.1, max_iter=20)

# Training loop with L-BFGS
epochs = 250
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()  # Clear previous gradients
        y_pred = model(X_train_torch)
        
        # Compute MSE loss
        mse_loss = criterion(y_pred, y_train_torch)
        rmse_loss = torch.sqrt(mse_loss)
        
        # Compute L1 and L2 loss
        l1_lambda = 0.001
        l2_lambda = 0.001
        l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        l2_loss = sum(torch.sum(param ** 2) for param in model.parameters())
        
        # Total loss with regularization
        total_loss = mse_loss + l2_lambda * l2_loss
        
        total_loss.backward()  # Compute gradients
        return total_loss

    optimizer.step(closure)  # Perform L-BFGS step

    if epoch % 50 == 0:
        with torch.no_grad():
            y_pred_train = model(X_train_torch)
            train_mse_loss = criterion(y_pred_train, y_train_torch)
            train_rmse_loss = torch.sqrt(train_mse_loss)
            print(f"Epoch {epoch}, Train RMSE: {train_rmse_loss.item()}")

# Evaluate model on test data
with torch.no_grad():
    y_test_pred = model(X_test_torch)
    test_mse_loss = criterion(y_test_pred, y_test_torch)
    test_rmse_loss = torch.sqrt(test_mse_loss)
    print(f"Test RMSE: {test_rmse_loss.item()}")


Epoch 0, Train RMSE: 255189008384.0
Epoch 50, Train RMSE: 92527.921875
Epoch 100, Train RMSE: 87112.78125
Epoch 150, Train RMSE: 86264.5546875
Epoch 200, Train RMSE: 84034.2734375
Test RMSE: 20970590.0
